# Unittest module

`unittest` is the built-in python module for unit testing.


### Unit test basics

A unit test suite in Python is represented by a file with the `test_` prefix defining a class extending `unittest.TestCase`. &nbsp;  
A unit test is represented by a method of this class with the `test_` prefix.  
In a unit test, we can use some assertion methods inherited from the `TestCase` class :
- self.assertEquals(a, b)
- self.assertTrue(a)
- self.assertFalse(a)
- self.assertRaises(err)
- self.assertIn(a, list)
-...



In [ ]:
import unittest

# import the module containing functions or classes to test
from calc import add


class TestCalculation(unittest.TestCase):
    
    def test_add(self):
        self.assertEquals(add(1, 2), 3)
        self.assertEquals(add(1, -2), -1)

    def test_divide(self):
        self.assertEquals(divide(1, 2), 0.5)
        with self.assertRaises(ValueError):
            divide(10, 0)


# Allow to run the unit test directly
if __name__ == '__main__':
    unittest.main()

### Setup and teardown

We can override the `setUp(self)` and `tearDown(self)` methods to execute some code before or after each test function.

We can override the `setUpClass(cls)` and `tearDownClass(cls)` class methods to execute some code once before or after all test functions.


In [ ]:
    @classmethod
    def setUpClass(cls):
        print('Called once before starting to run the tests')

    @classmethod
    def tearDownClass(cls):
        print('Called once after finishing to run the tests')

    def setUp(self):
        print('Called before every single test')
        self.calculator = Calculator(0, 0)

    def tearDown(self):
        print('Called after every single test')

### Mocking objects in unit tests

We can use the `patch()` function in `unittest.mock` to mock a function called in the code to test.  
This can be used to mock the response of an HTTP call or to an external module.

For example if the function to test uses the `requests` module to get an HTTP response :

In [ ]:
class Calculator:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def process(self):
        # HTTP call that we want to mock in the tests
        response = requests.get(f'http://calculator.com/process?a={self.a}&b={self.b}')
        if response.ok:
            return response.text
        else:
            return None

The call to `requests.get(url)` can be mocked in the test with :

In [ ]:
    def test_process(self):
        # Mock the call to requests.get() in the calculator class to simulate a success
        with patch('calc.requests.get') as mocked_get:
            mocked_get.return_value.ok = True
            mocked_get.return_value.text = '1234'

            calculator = Calculator(1, 2)
            self.assertEqual(calculator.process(), '1234')
            mocked_get.assert_called_with('http://calculator.com/process?a=1&b=2')

        # Mock the call to requests.get() in the calculator class to simulate a failure
        with patch('calc.requests.get') as mocked_get:
            mocked_get.return_value.ok = False

            calculator = Calculator(1, 2)
            self.assertEqual(calculator.process(), None)
            mocked_get.assert_called_with('http://calculator.com/process?a=1&b=2')

### Running the tests

To run the unit test suite, we can use the command : `python3 -m unittest test_calc.py`

If we run it without a test file specified, it will try to discover the test files : `python3 -m unittest`

We can also run a test file directly with `python3 test_calc.py` if we add at the end the block to run the unittest `main()` method when ran directly.

The default output is very concise :

In [ ]:
$> python3 -m unittest    
...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK

We can get a more verbose output with the `-v` flag :

In [ ]:
$> python3 -m unittest -v
test_add (test_calc.TestCalc) ... ok
test_divide (test_calc.TestCalc) ... ok
test_process (test_calc.TestCalc) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK